En este ejercicio vamos a partir de un data set muy conocido como es MNIST, el cual busca clasificar imágenes de los digitos codificados a mano. Este conjunto cuenta con 70,000 imagenes de 28x28, por lo que cada uno de los ejemplos cuenta con 784 varibles. pertenecientes a una de las 10 clases ( digitos del 0 al 9). Para cargarla podemos usar el siguiente código:

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import time


entradas, salidas = fetch_openml('mnist_784', version=1, return_X_y=True)

print(f"Patrones :{entradas.shape} -> {salidas.shape}")

/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Patrones :(70000, 784) -> (70000,)


Con esto se pide realizar las siguientes tareas:

1. Tras la carga de los datos anteriores, realizar una división entre entranamiento y test con el 80% de los datos para test y cuya semilla sea 42.

In [2]:

# Cargar los datos
entradas, salidas = fetch_openml('mnist_784', version=1, return_X_y=True)
print(f"Patrones :{entradas.shape} -> {salidas.shape}")

# Dividir los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(entradas, salidas, test_size=0.2, random_state=42)

# Imprimir las dimensiones de los conjuntos de entrenamiento y prueba
print(f"Entrenamiento: Entradas {X_train.shape}, Salidas {y_train.shape}")
print(f"Prueba: Entradas {X_test.shape}, Salidas {y_test.shape}")


/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Patrones :(70000, 784) -> (70000,)
Entrenamiento: Entradas (56000, 784), Salidas (56000,)
Prueba: Entradas (14000, 784), Salidas (14000,)


2. Definir un pipeline que cuente con una técnica de normalización o estandarización, 1 técnica de reducción de la dimensionalidad y una técnica de clasificación.

In [3]:

# Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),         # Estandarización
    ('pca', PCA(n_components=50)),        # Reducción de dimensionalidad
    ('classifier', RandomForestClassifier())  # Clasificación
])

# Entrenar el pipeline
pipeline.fit(X_train, y_train)

# Evaluar el pipeline en el conjunto de prueba
score = pipeline.score(X_test, y_test)
print(f"Precisión del pipeline: {score:.4f}")


Precisión del pipeline: 0.9441


3. Realizar el entrenamiento del pipeline un grid search para optimizar el resultado de la clasificación

In [4]:
# Cargar y dividir los datos
entradas, salidas = fetch_openml('mnist_784', version=1, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(entradas, salidas, test_size=0.2, random_state=42)

# Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('classifier', RandomForestClassifier())
])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'pca__n_components': [30, 50, 100],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 30],
    'classifier__min_samples_split': [2, 5]
}

# Crear y entrenar el Grid Search
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el Grid Search en el conjunto de prueba
score = grid_search.score(X_test, y_test)
print(f"Precisión del Grid Search: {score:.4f}")


/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Mejores hiperparámetros encontrados:
{'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200, 'pca__n_components': 100}
Precisión del Grid Search: 0.9470


4. Entrenar un pipeline con los valores óptimos del paso anterior y calcular las medidas oportunas de rendimiento y calcular el tiempo que tarda en ejecutarse.

In [5]:
# Cargar y dividir los datos
entradas, salidas = fetch_openml('mnist_784', version=1, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(entradas, salidas, test_size=0.2, random_state=42)

# Definir el pipeline con los valores óptimos
pipeline_optimizado = Pipeline([
('scaler', StandardScaler()),
('pca', PCA(n_components=100)),
('classifier', RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=2))
])

# Entrenar el pipeline y medir el tiempo de ejecución
start_time = time.time()
pipeline_optimizado.fit(X_train, y_train)
end_time = time.time()
tiempo_entrenamiento = end_time - start_time

# Evaluar el pipeline en el conjunto de prueba
score = pipeline_optimizado.score(X_test, y_test)

# Calcular las métricas de rendimiento
from sklearn.metrics import classification_report, confusion_matrix
y_pred = pipeline_optimizado.predict(X_test)
report = classification_report(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print(f"Precisión del pipeline optimizado: {score:.4f}")
print(f"Tiempo de entrenamiento: {tiempo_entrenamiento:.2f} segundos")
print("\nInforme de clasificación:")
print(report)
print("Matriz de confusión:")
print(confusion_mat)


/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Precisión del pipeline optimizado: 0.9471
Tiempo de entrenamiento: 127.51 segundos

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1343
           1       0.98      0.98      0.98      1600
           2       0.93      0.95      0.94      1380
           3       0.92      0.94      0.93      1433
           4       0.94      0.95      0.95      1295
           5       0.95      0.93      0.94      1273
           6       0.96      0.97      0.97      1396
           7       0.95      0.95      0.95      1503
           8       0.95      0.90      0.93      1357
           9       0.92      0.91      0.92      1420

    accuracy                           0.95     14000
   macro avg       0.95      0.95      0.95     14000
weighted avg       0.95      0.95      0.95     14000

Matriz de confusión:
[[1308    0    8    1    0    4   14    3    4    1]
 [   0 1573   10    4    1    1    2    2    4    3]
 [  